In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


# Helium 

## Initialize matrix parser

In [ ]:
from matrixelementparser import MatrixElementParser

Z = 2
matr = MatrixElementParser(Z=Z)
matr.show()

## Find reference energy

In [ ]:
def Eref(Z):
    Eref = 0
    for i in range(Z):
        for j in range(Z):
            Eref += 0.5*matr[i, j, i, j]
        Eref += matr.onebody(i, i)
    return Eref

[print(Eref(z)) for z in range(1,4)]

## Define hamiltonian up to singles-singles

In [ ]:
n_states = 5

F = 2
core = np.arange(F)
n_virtual = 4
virtual = np.arange(n_virtual) + F

refs = [0,1]
nref = len(refs) # this code needs a bit of rework if the ground state is degenerate

singles = [] # [(0,2),(1,3),(0,4),(1,5)]

for i in core:
    for a in virtual:
        if (i%2) == (a%2):
            singles.append((i, a))
            
nsingles = len(singles)

ref_ref = np.zeros((1,1))
ref_sing = np.zeros((nsingles,1))
sing_sing = np.zeros((nsingles,nsingles))

ref_ref[0,0] = Eref(Z)
for ind1, (i, a) in enumerate(singles):
    for ind2, (j, b) in enumerate(singles):
        E = matr[a, j, i, b] 
        E += matr.onebody(a, b) if i==j else 0
        E -= matr.onebody(i, j) if a==b else 0
        E += Eref(Z) if (i==j and a==b) else 0
        sing_sing[ind1, ind2] = E
    
    ref_sing[ind1] = sum(matr[i, j, a, j] for j in core) + matr.onebody(i, a)

                    
H = np.block([[ref_ref, ref_sing.T],
              [ref_sing, sing_sing]])

e,v = np.linalg.eig(H)
print(H)
print('eigenvalues')
print(e)

## Generalize code above into HartreeFock class

In [ ]:
from hartreefock import HartreeFock

system = HartreeFock(Z=2)
print(system.Hamiltonian())

print(system.energy_states()[0])